In [2]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

## Shufoo

In [2]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190219


In [3]:
xd_table = 'segment.drawbridge_device'

segment ALL

In [14]:
%%slack_notify @yusuke_akada
#XD拡張

segment_id = "sf0001","sf0002" #ALL

#f-string
query = (
    f"""
    with tmp as (
    select 
    distinct xd2.smn_uid
    from segment.shufoo_segment sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in (
     'iphone'
    ,'android_phone'
    ,'windows_phone'
    ,'windows_computer'
    ,'linux_computer'
    ,'mac'
    ,'ipad'
    ,'android_tablet'
    ,'windows_tablet') 
    and segment_id in {segment_id}
    )
    select 
    distinct smn_uid
    from tmp
    where smn_uid not in (select smn_uid from segment.shufoo_segment)
    
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    w

In [15]:
df_fix.shape[0]

23174579

In [17]:
df_final = df_fix[0:10000000]

In [19]:
df_final.shape[0]

10000000

In [20]:
print(df_final.dtypes)

smn_uid    int64
dtype: object


In [22]:
#HC3 CSVをUP
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    ir.upload_dataframe(f"segment.xd_shufoo_all", df_final, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Create empty 

segment その他もろもろ

In [8]:
%%slack_notify @yusuke_akada

xd_table = 'segment.drawbridge_device_1901'

#XD拡張
for segment_id in ['sf0001','sf0002','sf0120','sf0210','sf0220','sf0310','sf0320','sf0330','sf0340','sf0410','sf0420','sf0430','sf0510','sf0520']:
#for segment_id in ['sf0110']:
    query = (
        f"""
        with tmp as (
        select 
        distinct xd2.smn_uid
        from segment.shufoo_segment sd
        left join {xd_table} xd
        on sd.smn_uid = xd.smn_uid
        left join {xd_table} xd2
        on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
        where xd2.device in (
         'iphone'
        ,'android_phone'
        ,'windows_phone'
        ,'windows_computer'
        ,'linux_computer'
        ,'mac'
        ,'ipad'
        ,'android_tablet'
        ,'windows_tablet') 
        and segment_id in ('{segment_id}')
        )
        select 
        distinct smn_uid
        from tmp
        where smn_uid not in (select smn_uid from segment.shufoo_segment )

        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_fix = ir.sql_to_pandas(query)
        df_final = df_fix[0:10000000]
        ir.upload_dataframe(f"segment.xd_shufoo_{segment_id}", df_final, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [12]:
%%slack_notify @yusuke_akada

seed_table = 'segment.shufoo_segment'
xd_table = 'segment.drawbridge_device_1901'

#XD拡張
for segment_id in ['sf0110','sf0120','sf0210','sf0220','sf0310','sf0320','sf0330','sf0340','sf0410','sf0420','sf0430','sf0510','sf0520']:

    query = (
        f"""
        select 
        '{segment_id}' as segment_id
        ,count(distinct smn_uid) as uu
        from segment.xd_shufoo_{segment_id}
        group by 1
        
        """
    )
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        gs = GSheet()
        url = "https://docs.google.com/spreadsheets/d/1BkN7nGKW5twslsIYNIvz8s610iYgGp3TLASvvGgRHsA/edit#gid=0"
        sheet_index = 0
        col_offset=0
        gs.append_data(url, sheet_index, df, col_offset)


INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

↑バラバラのテーブルを作るとやはりのちのち面倒なので、一つのテーブルにする。なので削除

In [17]:
%%slack_notify @yusuke_akada
#テーブル削除
for segment_id in ['sf0110','sf0120','sf0210','sf0220','sf0310','sf0320','sf0330','sf0340','sf0410','sf0420','sf0430','sf0510','sf0520']:
    tables = [f"segment.xd_shufoo_{segment_id}"] 
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        ir.drop_tables(tables)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Dropping tabl

In [ ]:
%%slack_notify @yusuke_akada
xd_table = 'segment.drawbridge_device_1901'

uid_table = "segment.xd_shufoo_segment"   # segment_idとsmn_uidの最終的な対応table
temp_table = "dm_tmp.ya_temp_xd_shufoo_190220"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            smn_uid bigint
            )
        
        partitioned by 
            (
            segment_id string
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    
    
    #XD拡張 ただし、オリジナルのShufooセグメントにはいないユーザーで構成する
    
#    for segment_id in ['sf0120','sf0210','sf0220','sf0310','sf0320','sf0330','sf0340','sf0410','sf0420','sf0430','sf0510','sf0520']:
    for segment_id in ['sf0210']:
        #for segment_id in ['sf0110']:
        query = (
            f"""
            with tmp as (
            select 
            distinct xd2.smn_uid
            from segment.shufoo_segment sd
            left join {xd_table} xd
            on sd.smn_uid = xd.smn_uid
            left join {xd_table} xd2
            on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
            where xd2.device in (
             'iphone'
            ,'android_phone'
            ,'windows_phone'
            ,'windows_computer'
            ,'linux_computer'
            ,'mac'
            ,'ipad'
            ,'android_tablet'
            ,'windows_tablet') 
            and segment_id in ('{segment_id}')
            )
            select 
            distinct smn_uid
            from tmp
            where smn_uid not in (select smn_uid from segment.shufoo_segment )
            """
        )
        
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            
            df = ir.sql_to_pandas(query)
            
            # ImpalaにUP
            ir.upload_dataframe(temp_table, df, ['bigint'])
        
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    smn_uid 
                    )

                partition
                    (
                    segment_id = '{segment_id}'
                    )

                select
                 smn_uid
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])

In [49]:

uid_table = "dm_tmp.ya_xd_shufoo_segment"   # segment_idとsmn_uidの最終的な対応table


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            smn_uid bigint
            )
        
        partitioned by 
            (
            segment_id string
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.6/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [4]:
%%slack_notify @yusuke_akada
xd_table = 'segment.drawbridge_device_1901'

uid_table = "dm_tmp.ya_xd_shufoo_segment"   # segment_idとsmn_uidの最終的な対応table
temp_table = "dm_tmp.ya_temp_xd_shufoo_190304"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    
    #XD拡張 ただし、オリジナルのShufooセグメントにはいないユーザーで構成する
    
#    for segment_id in ['sf0110','sf0120','sf0210','sf0220','sf0310','sf0320','sf0330','sf0340','sf0410','sf0420','sf0430','sf0510','sf0520']:
    for segment_id in ['sf0001','sf0002']:
        query = (
            f"""
            with tmp as (
            select 
            distinct xd2.smn_uid
            from segment.shufoo_segment sd
            left join {xd_table} xd
            on sd.smn_uid = xd.smn_uid
            left join {xd_table} xd2
            on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
            where xd2.device in (
             'iphone'
            ,'android_phone'
            ,'windows_phone'
            ,'windows_computer'
            ,'linux_computer'
            ,'mac'
            ,'ipad'
            ,'android_tablet'
            ,'windows_tablet') 
            and segment_id in ('{segment_id}')
            )
            select 
            distinct smn_uid
            from tmp
            where smn_uid not in (select smn_uid from segment.shufoo_segment where segment_id not in ('{segment_id}'))
            """
        )
        
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            
            df = ir.sql_to_pandas(query)
            
            # ImpalaにUP
            ir.upload_dataframe(temp_table, df, ['bigint'])
        
        
        # 親テーブルにInsert
            query = (
                f"""
                insert overwrite table {uid_table}
                    (
                    smn_uid 
                    )

                partition
                    (
                    segment_id = '{segment_id}'
                    )

                select
                 smn_uid
                from {temp_table}
                """
            )
            ir.execute_query_no_ret(query) 

            # tempテーブルdrop
            ir.drop_tables([temp_table])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.smn.impala_clien